<a href="https://colab.research.google.com/github/alyssa-blair/csc-485B-group8/blob/main/csc485b_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-_hpxp9ni
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-_hpxp9ni
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-lg4djo7u/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp8vy9aib7".


In [ ]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <stdint.h> // uint32_t

using element_t = uint32_t;

In [ ]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [ ]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <random>  // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {

/**
 * Generates and returns a vector of random uniform data of a given length, n,
 * for any integral type. Input range will be [0, 2n].
 */
template < typename T >
std::vector< T > generate_uniform( std::size_t n )
{
    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20240916;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 2 * n);

    std::vector< T > random_data( n ); // init array
    std::generate( std::begin( random_data )
                 , std::end  ( random_data )
                 , [ &rng, &distrib ](){ return static_cast< T >( distrib( rng ) ); });

    return random_data;
}

} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "algorithm_choices.h"
#pragma once

#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {
namespace cpu {

void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace cpu


namespace gpu {

void run_gpu_soln( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "cpu_baseline.cu"
/**
 * CPU methods that the GPU should outperform.
 */

#include "algorithm_choices.h"

#include <algorithm> // std::sort()
#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

namespace csc485b {
namespace a1      {
namespace cpu     {

/**
 * Simple solution that just sorts the whole array with a built-in sort
 * function and then resorts the last portion in the opposing order with
 * a second call to that same built-in sort function.
 */
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    std::sort( data, data + num_elements, std::less< element_t >{} );
    std::sort( data + invert_at_pos, data + num_elements, std::greater< element_t >{} );
}

/**
 * Run the single-threaded CPU baseline that students are supposed to outperform
 * in order to obtain higher grades on this assignment. Times the execution and
 * prints to the standard output (e.g., the screen) that "wall time." Note that
 * the functions takes the input by value so as to not perturb the original data
 * in place.
 */
void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n )
{
    auto const cpu_start = std::chrono::high_resolution_clock::now();
    opposing_sort( data.data(), switch_at, n );
    auto const cpu_end = std::chrono::high_resolution_clock::now();

    std::cout << "CPU Baseline time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(cpu_end - cpu_start).count()
              << " ns" << std::endl;

    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
}

} // namespace cpu
} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "gpu_solution.cu"
/**
 * The file in which you will implement your GPU solutions!
 */

#include "algorithm_choices.h"

#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

#include "cuda_common.h"

namespace csc485b {
namespace a1      {
namespace gpu     {

/**
 * The CPU baseline benefits from warm caches because the data was generated on
 * the CPU. Run the data through the GPU once with some arbitrary logic to
 * ensure that the GPU cache is warm too and the comparison is more fair.
 */
__global__
void warm_the_gpu( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    // We know this will never be true, because of the data generator logic,
    // but I doubt that the compiler will figure it out. Thus every element
    // should be read, but none of them should be modified.
    if( th_id < num_elements && data[ th_id ] > num_elements * 100 )
    {
        ++data[ th_id ]; // should not be possible.
    }
}

/**
 * Your solution. Should match the CPU output.
 *
 * An implemnetation of the Bitonic Sort algorithm, where sorting order flips at invert_at_pos.
 */
__global__
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    std::size_t const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    // Check that the thread id is within the number of elements
    if( th_id < num_elements )
    {
        // Loop over the bitonic sequence length
        for ( std::size_t sequence_len = 2; sequence_len <= num_elements; sequence_len <<= 1 )
        {
            // Loop over step sizes for this sequence length
            for ( std::size_t step_size = sequence_len >> 1; step_size > 0; step_size >>= 1 )
            {
                // Calculate the position of the partner element
                std::size_t position = th_id ^ step_size;
                std::size_t sequence_num = th_id / sequence_len;

                // Check that this thread, and it's partner are in the same sequence
                if ( th_id < position && position < num_elements && sequence_num == (position / sequence_len) )
                {
                    // Determine sorting direction
                    bool dir = sequence_num & 1 ^ ( sequence_len == num_elements && num_elements / step_size >= 4 && th_id >= invert_at_pos );

                    // Check if the elements should be swapped
                    if ( (data[th_id] < data[position]) == dir )
                    {
                        element_t tmp = data[ th_id ];
                        data[ th_id ] = data[ position ];
                        data[ position ] = tmp;
                    }
                }
                __syncthreads();
            }
        }
        return;
    }
}

/**
 * Performs all the logic of allocating device vectors and copying host/input
 * vectors to the device. Times the opposing_sort() kernel with wall time,
 * but excludes set up and tear down costs such as mallocs, frees, and memcpies.
 */
void run_gpu_soln( std::vector< element_t > data, std::size_t switch_at, std::size_t n )
{
    // Kernel launch configurations. Feel free to change these.
    // This is set to maximise the size of a thread block on a T4, but it hasn't
    // been tuned. It's not known if this is optimal.
    std::size_t const threads_per_block = 1024;
    std::size_t const num_blocks =  ( n + threads_per_block - 1 ) / threads_per_block;

    // Allocate arrays on the device/GPU
    element_t * d_data;
    cudaMalloc( (void**) & d_data, sizeof( element_t ) * n );
    CHECK_ERROR("Allocating input array on device");

    // Copy the input from the host to the device/GPU
    cudaMemcpy( d_data, data.data(), sizeof( element_t ) * n, cudaMemcpyHostToDevice );
    CHECK_ERROR("Copying input array to device");

    // Warm the cache on the GPU for a more fair comparison
    warm_the_gpu<<< num_blocks, threads_per_block>>>( d_data, switch_at, n );

    // Time the execution of the kernel that you implemented
    auto const kernel_start = std::chrono::high_resolution_clock::now();
    opposing_sort<<< num_blocks, threads_per_block>>>( d_data, switch_at, n );
    auto const kernel_end = std::chrono::high_resolution_clock::now();
    CHECK_ERROR("Executing kernel on device");

    // After the timer ends, copy the result back, free the device vector,
    // and echo out the timings and the results.
    cudaMemcpy( data.data(), d_data, sizeof( element_t ) * n, cudaMemcpyDeviceToHost );
    CHECK_ERROR("Transferring result back to host");
    cudaFree( d_data );
    CHECK_ERROR("Freeing device memory");

    std::cout << "GPU Solution time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(kernel_end - kernel_start).count()
              << " ns" << std::endl;

    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
}

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [ ]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <cstddef>  // std::size_t type
#include <iostream> // std::cout, std::endl
#include <vector>

#include "algorithm_choices.h"
#include "data_generator.h"
#include "data_types.h"
#include "cuda_common.h"

int main()
{
    std::size_t const n = 1024;
    std::size_t const switch_at = 3 * ( n >> 2 ) ;

    auto data = csc485b::a1::generate_uniform< element_t >( n );
    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;

    csc485b::a1::cpu::run_cpu_baseline( data, switch_at, n );
    csc485b::a1::gpu::run_gpu_soln( data, switch_at, n );

    return EXIT_SUCCESS;
}

In [ ]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

40 728 413 1246 1646 1668 1888 6 341 1632 351 212 1004 1116 1160 428 421 1778 1088 13 621 266 53 872 1048 236 326 608 865 316 1806 480 863 679 1337 1072 83 1026 1986 730 295 928 488 305 464 972 576 1064 1220 122 1227 1329 285 49 742 1146 761 1455 471 202 763 1616 1187 1052 1584 711 1453 998 1497 1290 992 1518 1953 616 1506 1998 932 1228 831 1616 922 1769 1652 2045 560 1955 790 1444 1773 682 1985 679 1072 693 238 1663 1946 1586 575 468 274 1137 1519 1493 71 780 1538 212 1010 480 334 1331 131 1957 754 199 430 1408 471 1685 1298 141 1501 719 1533 1386 767 1491 591 514 575 689 254 1650 1172 695 810 329 1459 150 654 483 1535 2026 110 1424 1378 315 101 555 815 1924 71 1328 485 942 1465 1323 1249 1109 1301 276 743 1283 768 374 221 1657 238 1635 1342 1680 1573 1725 1512 939 844 881 595 442 1409 2029 1558 1586 508 2003 955 1551 1413 698 179 1250 1522 836 2000 599 925 981 2040 1793 1909 1994 1767 1782 1467 1609 1220 990 1808 1483 65 1535 384 213 67 68 790 1394 1261 104 1845 443 559 1344 620 1131